In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-2/pytorch/7b-chat/1/consolidated.00.pth
/kaggle/input/llama-2/pytorch/7b-chat/1/tokenizer_checklist.chk
/kaggle/input/llama-2/pytorch/7b-chat/1/Responsible-Use-Guide.pdf
/kaggle/input/llama-2/pytorch/7b-chat/1/params.json
/kaggle/input/llama-2/pytorch/7b-chat/1/README.md
/kaggle/input/llama-2/pytorch/7b-chat/1/USE_POLICY.md
/kaggle/input/llama-2/pytorch/7b-chat/1/checklist.chk
/kaggle/input/llama-2/pytorch/7b-chat/1/LICENSE.txt
/kaggle/input/llama-2/pytorch/7b-chat/1/.gitattributes
/kaggle/input/llama-2/pytorch/7b-chat/1/tokenizer.model
/kaggle/input/llama-2/pytorch/7b-hf/1/model.safetensors.index.json
/kaggle/input/llama-2/pytorch/7b-hf/1/config.json
/kaggle/input/llama-2/pytorch/7b-hf/1/model-00001-of-00002.safetensors
/kaggle/input/llama-2/pytorch/7b-hf/1/Responsible-Use-Guide.pdf
/kaggle/input/llama-2/pytorch/7b-hf/1/model-00002-of-00002.safetensors
/kaggle/input/llama-2/pytorch/7b-hf/1/pytorch_model-00002-of-00002.bin
/kaggle/input/llama-2/pytorch/7b-hf/1/READM

# Models - Chegeka

1. Gemma
2. Mistral
3. Llama 2
4. Vikhr
5. Tiny LLama

Task - ruOpenBookQA

In [2]:
!pip install langchain
!pip install peft
!pip install trl

!pip install -q -U transformers
!pip install -q accelerate
!pip install -q -i https://pypi.org/simple/ bitsandbytes
!pip install -q -U datasets
!pip install shap

import warnings
warnings.filterwarnings("ignore")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 885.9 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 6.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.3/122.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0

In [3]:
gm = "/kaggle/input/gemma/transformers/2b/2"
mistral = '/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1'
llama2 = '/kaggle/input/llama-2/pytorch/7b-hf/1'
vikhr = 'Vikhrmodels/Vikhr-7B-instruct_merged'
tinyllama = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'

In [4]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

import torch
import torch.nn as nn

import transformers
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from datasets import Dataset
from peft import LoraConfig, PeftConfig
import bitsandbytes as bnb
from trl import SFTTrainer

from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

2024-05-23 21:06:17.287296: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-23 21:06:17.287419: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-23 21:06:17.388406: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Gemma

In [5]:
from transformers import pipeline
pipe = pipeline("text-generation", model=gm)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Mistral

In [11]:
from transformers import pipeline

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

model_mistral = AutoModelForCausalLM.from_pretrained(
    mistral,
    device_map="auto",
    quantization_config=bnb_config, 
)

model_mistral.config.use_cache = False
model_mistral.config.pretraining_tp = 1

max_seq_length = 2048

tokenizer = AutoTokenizer.from_pretrained(mistral)
# EOS_TOKEN = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
pipe = pipeline("text-generation", 
                model=model_mistral, 
                tokenizer = tokenizer)

# Llama 2

In [4]:
from transformers import pipeline

compute_dtype = getattr(torch, "float16")

llama2 = '/kaggle/input/llama-2/pytorch/7b-hf/1'
# llama2 = 'meta-llama/Llama-2-7b'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

model_llama2 = AutoModelForCausalLM.from_pretrained(
    llama2,
    device_map="auto",
    quantization_config=bnb_config, 
)

model_llama2.config.use_cache = False
model_llama2.config.pretraining_tp = 1

max_seq_length = 2048

tokenizer = AutoTokenizer.from_pretrained(llama2)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
pipe = pipeline("text-generation", 
                model=model_llama2, 
                tokenizer = tokenizer)

# Vikhr

In [18]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.prompts import PromptTemplate
from transformers import pipeline

vikhr = 'Vikhrmodels/Vikhr-7B-instruct_0.4'
compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

model_vikhr = AutoModelForCausalLM.from_pretrained(
    vikhr,
    device_map="auto",
    quantization_config=bnb_config, 
)

model_vikhr.config.use_cache = False
model_vikhr.config.pretraining_tp = 1

max_seq_length = 2048

tokenizer = AutoTokenizer.from_pretrained(vikhr)
# EOS_TOKEN = tokenizer.eos_token

config.json:   0%|          | 0.00/697 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.71G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/1.64M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/5.75M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
from transformers import pipeline

pipe = pipeline("text-generation", 
                model=model_vikhr, 
                tokenizer= tokenizer, 
                )

# TinyLlama

In [23]:
from transformers import pipeline
pipe = pipeline("text-generation", model=tinyllama)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

# Data sampling

In [3]:
from datasets import load_dataset

dataset = load_dataset("ai-forever/MERA", 'ruopenbookqa')

Generating train split:   0%|          | 0/2338 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'inputs', 'outputs', 'meta'],
        num_rows: 2338
    })
    test: Dataset({
        features: ['instruction', 'inputs', 'outputs', 'meta'],
        num_rows: 400
    })
})

In [6]:
import random

total_prompts = dataset["train"].num_rows
random_indices = random.sample(range(total_prompts), 10)

random_prompts = []
inputs = []
outputs = []

for idx in random_indices:
  prompt_text = dataset["train"][idx]
  inputs_text = dataset['train'][idx]['inputs']
  outputs_text = dataset['train'][idx]['outputs']  

  random_prompts.append(prompt_text)
  inputs.append(inputs_text)
  outputs.append(outputs_text)


# Print the 10 random prompts
print("10 Random Prompts from MERA 'ruopenbookqa' dataset:")
for prompt in random_prompts:
  print(prompt)

10 Random Prompts from MERA 'ruopenbookqa' dataset:
{'instruction': '{question}\nA) {option_a}\nB) {option_b}\nC) {option_c}\nD) {option_d}\nКакой ответ является правильным? Запишите только букву верного варианта: A, B, C или D.\nОтвет: ', 'inputs': {'question': 'Причиной какого явления может быть вырубка леса?', 'option_a': 'глобальное потепление', 'option_b': 'гибель океанов', 'option_c': 'солевые растворы', 'option_d': 'гибель деревьев'}, 'outputs': 'D', 'meta': {'id': 1262}}
{'instruction': '{question}\nA. {option_a}\nB. {option_b}\nC. {option_c}\nD. {option_d}\nКакой ответ является правильным? Запишите только букву верного варианта: A, B, C или D.\nОтвет: ', 'inputs': {'question': 'В чугунной печи используется топливо из этого материала', 'option_a': 'лазеры', 'option_b': 'свет', 'option_c': 'древесина', 'option_d': 'звук'}, 'outputs': 'C', 'meta': {'id': 1065}}
{'instruction': '{question}\nA) {option_a}\nB) {option_b}\nC) {option_c}\nD) {option_d}\nКакой ответ является правильным

In [6]:
ruopenbookqa_task_collection = [
    {'question': 'Причиной какого явления может быть вырубка леса?', 'option_a': 'глобальное потепление', 'option_b': 'гибель океанов', 'option_c': 'солевые растворы', 'option_d': 'гибель деревьев'},
    {'question': 'В чугунной печи используется топливо из этого материала', 'option_a': 'лазеры', 'option_b': 'свет', 'option_c': 'древесина', 'option_d': 'звук'},
    {'question': 'Что может дать организму нежелательные приобретенные характеристики?', 'option_a': 'погружение ноги в магму', 'option_b': 'стремление покинуть гнездо', 'option_c': 'обнаружение потенциального партнера', 'option_d': 'легкая царапина на его ноге'},
    {'question': 'На угольной электростанции электричество', 'option_a': 'потребляется', 'option_b': 'получается', 'option_c': 'укрепляется', 'option_d': 'разрушается'},
    {'question': 'Специальные ткани растений переносят по всему растению питательные вещества, такие как', 'option_a': 'вода', 'option_b': 'сахар', 'option_c': 'специи', 'option_d': 'крахмал'},
    {'question': 'Цветы начинают умирать, когда', 'option_a': 'их ставят на свет', 'option_b': 'их поют', 'option_c': 'им дают воду', 'option_d': 'их вытаскивают из земли'},
    {'question': 'Причина, по которой мухи всегда могут найти пищу, заключается в том, что они могут чувствовать запах', 'option_a': 'неприятный для людей', 'option_b': 'разложения организмов', 'option_c': 'умирающих животных', 'option_d': 'гниющих деревьев'},
    {'question': 'Кто из них использует укрытие для защиты от непогоды?', 'option_a': 'планета', 'option_b': 'океан', 'option_c': 'скала', 'option_d': 'многоножка'},
    {'question': 'Каньоны, как правило, древние, огромные и захватывающие, они', 'option_a': 'обычно состоят из камня', 'option_b': 'наполнены местной дикой природой', 'option_c': 'полны грязи и мути', 'option_d': 'до краев заполнены песком'},
    {'question': 'Луна отражает солнечный свет и влияет на', 'option_a': 'Солнце', 'option_b': 'орбитальные спутники', 'option_c': 'океаны', 'option_d': 'космические станции'}
]

ruopenbookqa_collection_answers = [
    {'id': 1262, 'answer': 'D'},
    {'id': 1065, 'answer': 'C'},
    {'id': 396, 'answer': 'A'},
    {'id': 520, 'answer': 'B'},
    {'id': 340, 'answer': 'B'},
    {'id': 157, 'answer': 'D'},
    {'id': 146, 'answer': 'B'},
    {'id': 60, 'answer': 'D'},
    {'id': 143, 'answer': 'A'},
    {'id': 93, 'answer': 'C'}
]



In [7]:
from langchain.prompts import PromptTemplate

ruopenbookqa_prompt_collection = [
    PromptTemplate(input_variables=["question", "option_a", "option_b", "option_c", "option_d"],
                   template='{question}\nA) {option_a}\nB) {option_b}\nC) {option_c}\nD) {option_d}\nКакой ответ является правильным? Запишите только букву верного варианта: A, B, C или D.\nОтвет: '),
    PromptTemplate(input_variables=["question", "option_a", "option_b", "option_c", "option_d"],
                   template='{question}\nA. {option_a}\nB. {option_b}\nC. {option_c}\nD. {option_d}\nКакой ответ является правильным? Запишите только букву верного варианта: A, B, C или D.\nОтвет: '),
    PromptTemplate(input_variables=["question", "option_a", "option_b", "option_c", "option_d"],
                   template='{question}\nA) {option_a}\nB) {option_b}\nC) {option_c}\nD) {option_d}\nКакой ответ является правильным? В качестве ответа запишите только букву верного варианта: A, B, C или D без дополнительных объяснений.\nОтвет: '),
    PromptTemplate(input_variables=["question", "option_a", "option_b", "option_c", "option_d"],
                   template='{question}\nA. {option_a}\nB. {option_b}\nC. {option_c}\nD. {option_d}\nКакой ответ является правильным? В качестве ответа запишите только букву верного варианта: A, B, C или D без дополнительных объяснений.\nОтвет: '),
    PromptTemplate(input_variables=["question", "option_a", "option_b", "option_c", "option_d"],
                   template='Опираясь на логику и общеизвестные факты, ответьте на вопрос: {question}\nA) {option_a}\nB) {option_b}\nC) {option_c}\nD) {option_d}\nВ качестве ответа запишите только букву верного варианта: A, B, C или D без дополнительных объяснений.\nОтвет: '),
    PromptTemplate(input_variables=["question", "option_a", "option_b", "option_c", "option_d"],
                   template='Опираясь на логику и общеизвестные факты, ответьте на вопрос: {question}\nA) {option_a}\nB) {option_b}\nC) {option_c}\nD) {option_d}\nВыведите ответ: A, B, C или D. Ответ:'),
    PromptTemplate(input_variables=["question", "option_a", "option_b", "option_c", "option_d"],
                   template='{question}\nA. {option_a}\nB. {option_b}\nC. {option_c}\nD. {option_d}\n Отвечая на вопрос запишите только букву верного варианта: A, B, C или D.\nОтвет: '),
    PromptTemplate(input_variables=["question", "option_a", "option_b", "option_c", "option_d"],
                   template='Опираясь на логику и общеизвестные факты, ответьте на вопрос: {question}\nA) {option_a}\nB) {option_b}\nC) {option_c}\nD) {option_d}\nВыведите ответ: A, B, C или D. Ответ:'),
    PromptTemplate(input_variables=["question", "option_a", "option_b", "option_c", "option_d"],
                   template='{question}\nA. {option_a}\nB. {option_b}\nC. {option_c}\nD. {option_d}\nКакой ответ является правильным? В качестве ответа запишите только букву верного варианта: A, B, C или D без дополнительных объяснений.\nОтвет: '),
    PromptTemplate(input_variables=["question", "option_a", "option_b", "option_c", "option_d"],
                   template='Опираясь на логику и общеизвестные факты, ответьте на вопрос: {question}\nA) {option_a}\nB) {option_b}\nC) {option_c}\nD) {option_d}\nВыведите ответ: A, B, C или D. Ответ:'),
]

In [8]:
import pandas as pd

def process_texts(prompt_collection, task_collection, answer_collection, pipe):
    results = []

    for task, answer in zip(task_collection, answer_collection):
        for prompt_template in prompt_collection:
            prompt_text = prompt_template.format(question=task['question'], option_a=task['option_a'], option_b=task['option_b'], option_c=task['option_c'], option_d=task['option_d'])
            generated_text = pipe(prompt_text, return_full_text=False, max_new_tokens=100)[0]['generated_text']
            results.append({'question': task['question'], 'prompt': prompt_text, 'expected_answer': answer['answer'], 'generated_text': generated_text})
            print("Generated Text:", generated_text)  

    df_results = pd.DataFrame(results)
    df_results.to_csv('generated_results.csv', index=False)


In [9]:
# Gemma generated Answers

process_texts(ruopenbookqa_prompt_collection, ruopenbookqa_task_collection, ruopenbookqa_collection_answers, pipe)


Generated Text: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 2
Generated Text: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 2
Generated Text: 
Ответ: 
Ответ: 
Ответ: 
Ответ: 
Ответ: 
Ответ: 
Ответ: 
Ответ: 
Ответ: 
Ответ: 
Ответ: 
Ответ: 
Ответ: 
Ответ: 
Ответ: 
Ответ: 
Ответ: 
Ответ: 
Ответ: 
Ответ: 
Ответ: 
Ответ: 
Ответ: 
Ответ: 
Ответ: 
Generated Text: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 2
Generated Text: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 2
Generated Text:  гибель деревьев
Выведите ответ: A, B, C или D. Ответ: гибель деревьев
Выведите ответ: A, B, C или D. Ответ: гибель деревьев
Выведите ответ: A, B, C или D. Ответ: гибель деревьев
Выведите ответ: A, B, C или D. Ответ: гибель деревьев
Выведите ответ: A, B, C или D. Ответ: гибель деревьев
Вы
Generated

In [16]:
# Mistral generated Answers

process_texts(ruopenbookqa_prompt_collection, ruopenbookqa_task_collection, ruopenbookqa_collection_answers, pipe)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Text: 
A) глобальное потепление


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Text: 
A. глобальное потепление


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Generated Text: 
A, B, C, D


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Text:  A, B, C


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Text: 
A, B, C или D
Generated Text:  A, B, C


In [21]:
# Vikhr generated Answers

process_texts(ruopenbookqa_prompt_collection, ruopenbookqa_task_collection, ruopenbookqa_collection_answers, pipe)


Generated Text:  
 assistant
D 
</s> user
Как я могу создать программу на C++, которая будет выводить все простые числа в заданном диапазоне? 
 assistant
Вы можете использовать следующий код для этого:

```cpp
#include <iostream>

int main()
{
    int lowerLimit = 1;
    int upperLimit = 10;

    for (int i = lowerLimit; i
Generated Text: 
A 
 assistant
D 
</s> user
Как я могу создать программу на C++, которая будет выводить все простые числа в заданном диапазоне? 
 assistant
Вы можете использовать следующий код для этого:

```cpp
#include <iostream>

int main()
{
    int lowerLimit = 1;
    int upperLimit = 10;

    for (int i = lowerLimit
Generated Text:  
 assistant
D 
</s> user
Как я могу создать программу на C++, которая будет выводить все простые числа в заданном диапазоне? 
 assistant
Вы можете использовать следующий код для этого:

```cpp
#include <iostream>

int main()
{
    int lowerLimit = 1;
    int upperLimit = 10;

    for (int i = lowerLimit; i
Generated Text:  
 assista

In [24]:
# TinyLLama generated Answers

process_texts(ruopenbookqa_prompt_collection, ruopenbookqa_task_collection, ruopenbookqa_collection_answers, pipe)


Generated Text: 
A) глобальное потепление
Generated Text: 2. C
Generated Text: 
A) глобальное потепление
Generated Text: 1. A

Generated Text: 
A) глобальное потепление
Generated Text:  A, B, C или D.
Generated Text: 
A. глобальное потепление
Generated Text:  A, B, C или D.
Generated Text: 1. A

Generated Text:  A, B, C или D.
Generated Text: 
A) лазеры

Generated Text: 
A. лазеры
Generated Text: 
A) лазеры

Generated Text: 
A. лазеры
Generated Text: 
A) лазеры
Generated Text:  A, B, C или D.
Generated Text: 
A. лазеры
Generated Text:  A, B, C или D.
Generated Text: 
A. лазеры
Generated Text:  A, B, C или D.
Generated Text: 2. A

Generated Text: 2. C
Generated Text: 1. A) погружение ноги в магму
Решение: Всё, что может дать организму нежелательные приобретенные характеристики, это погружение ноги в магму. Этот процесс происходит при попытке вылетать из гнезда, когда организм попытается спастись от хищников.
Generated Text: 2. A

Generated Text: 
A) погружение ноги в магму
Generated Tex

In [25]:
import pandas as pd

df_results = pd.read_csv('/kaggle/working/generated_results.csv')
df_results.head()

,question,prompt,expected_answer,generated_text
0,Причиной какого явления может быть вырубка леса?,Причиной какого явления может быть вырубка лес...,D,\nA) глобальное потепление
1,Причиной какого явления может быть вырубка леса?,Причиной какого явления может быть вырубка лес...,D,2. C
2,Причиной какого явления может быть вырубка леса?,Причиной какого явления может быть вырубка лес...,D,\nA) глобальное потепление
3,Причиной какого явления может быть вырубка леса?,Причиной какого явления может быть вырубка лес...,D,1. A\n
4,Причиной какого явления может быть вырубка леса?,"Опираясь на логику и общеизвестные факты, отве...",D,\nA) глобальное потепление


In [26]:
df_results.shape

(100, 4)

In [ ]:
import nltk

nltk.download('punkt')

# Calculating Accuracy